In [1]:
import inspect

# Accelerate arithmetic by accepting probabilistic proofs
from sage.structure.proof.all import arithmetic
arithmetic(False)

def derive_BN_field(x):
  params = {
    'param': x,
    'modulus': 36*x^4 + 36*x^3 + 24*x^2 + 6*x + 1,
    'order': 36*x^4 + 36*x^3 + 18*x^2 + 6*x + 1,
    'trace': 6*x^2 + 1,
    'family': 'BN'
  }
  return params

def derive_BLS12_field(x):
  params = {
    'param': x,
    'modulus': (x - 1)^2 * (x^4 - x^2 + 1)//3 + x,
    'order': x^4 - x^2 + 1,
    'trace': x + 1,
    'family': 'BLS12'
  }
  return params

def derive_BW6_compose_BLS12_field(x, cofactor_trace, cofactor_y):
  # Brezing-Weng input
  r = (x^6 - 2*x^5 + 2*x^3 + x + 1) // 3 # BLS12 modulus

  # 6-th root of unity output + cofactors
  t = x^5 - 3*x^4 + 3*x^3 - x + 3 + cofactor_trace*r
  y = (x^5 - 3*x^4 + 3*x^3 - x + 3)//3 + cofactor_y*r

  # Curve parameters
  p = (t^2 + 3*y^2)/4
  trace = p+1-r # (3*y+t)/2

  params = {
    'param': x,
    'modulus': p,
    'order': r,
    'trace': trace,
    'family': 'BW6'
  }
  return params

def copyright():
  return inspect.cleandoc("""
    # Constantine
    # Copyright (c) 2018-2019    Status Research & Development GmbH
    # Copyright (c) 2020-Present Mamy André-Ratsimbazafy
    # Licensed and distributed under either of
    #   * MIT license (license terms in the root directory or at http://opensource.org/licenses/MIT).
    #   * Apache v2 license (license terms in the root directory or at http://www.apache.org/licenses/LICENSE-2.0).
    # at your option. This file may not be copied, modified, or distributed except according to those terms.
  """)

Curves = {
  'BN254_Nogami': {
    'field': derive_BN_field(-(2^62 + 2^55 + 1)),
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 2
    },
    'tower': {
      'embedding_degree': 12,
      'twist_degree': 6,
      'QNR_Fp': -1,
      'SNR_Fp2': [1, 1],
      'twist': 'D_Twist'
    }
  },
  'BN254_Snarks': {
    'field': derive_BN_field(Integer('0x44e992b44a6909f1')),
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 3
    },
    'tower': {
      'embedding_degree': 12,
      'twist_degree': 6,
      'QNR_Fp': -1,
      'SNR_Fp2': [9, 1],
      'twist': 'D_Twist'
    }
  },
  'BLS12_377': {
    'field': derive_BLS12_field(3 * 2^46 * (7 * 13 * 499) + 1),
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 1
    },
    'tower': {
      'embedding_degree': 12,
      'twist_degree': 6,
      'QNR_Fp': -5,
      'SNR_Fp2': [0, 1],
      'twist': 'D_Twist'
    }
  },
  'BLS12_381': {
    'field': derive_BLS12_field(-(2^63 + 2^62 + 2^60 + 2^57 + 2^48 + 2^16)),
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 4
    },
    'tower': {
      'embedding_degree': 12,
      'twist_degree': 6,
      'QNR_Fp': -1,
      'SNR_Fp2': [1, 1],
      'twist': 'M_Twist'
    }
  },
  'BW6_761': {
    'field': derive_BW6_compose_BLS12_field(
        3 * 2^46 * (7 * 13 * 499) + 1,
        cofactor_trace = 13,
        cofactor_y = 9
    ),
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': -1
    },
    'tower': {
      'embedding_degree': 6,
      'twist_degree': 6,
      'SNR_Fp': -4,
      'twist': 'M_Twist'
    }
  },
  'Pallas': {
    'field': {
      'modulus': Integer('0x40000000000000000000000000000000224698fc094cf91b992d30ed00000001'),
      'order': Integer('0x40000000000000000000000000000000224698fc0994a8dd8c46eb2100000001')
    },
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 5
    }
  },
  'Vesta': {
    'field': {
      'modulus':  Integer('0x40000000000000000000000000000000224698fc0994a8dd8c46eb2100000001'),
      'order': Integer('0x40000000000000000000000000000000224698fc094cf91b992d30ed00000001'),
    },
    'curve': {
      'form': 'short_weierstrass',
      'a': 0,
      'b': 5
    }
  }
}


In [19]:
def genScalarMulG2(curve_name, curve_config, count, seed, scalarBits = None):
  p = curve_config[curve_name]['field']['modulus']
  r = curve_config[curve_name]['field']['order']
  form = curve_config[curve_name]['curve']['form']
  a = curve_config[curve_name]['curve']['a']
  b = curve_config[curve_name]['curve']['b']
  embedding_degree = curve_config[curve_name]['tower']['embedding_degree']
  twist_degree = curve_config[curve_name]['tower']['twist_degree']
  twist = curve_config[curve_name]['tower']['twist']


  G2_field_degree = embedding_degree // twist_degree
  G2_field = f'Fp{G2_field_degree}' if G2_field_degree > 1 else 'Fp'


  if G2_field_degree == 2:
    non_residue_fp = curve_config[curve_name]['tower']['QNR_Fp']
  elif G2_field_degree == 1:
    if twist_degree == 6:
      # Only for complete serialization
      non_residue_fp = curve_config[curve_name]['tower']['SNR_Fp']
    else:
      raise NotImplementedError()
  else:
    raise NotImplementedError()


  Fp = GF(p)
  K.<u> = PolynomialRing(Fp)


  if G2_field == 'Fp2':
    Fp2.<beta> = Fp.extension(u^2 - non_residue_fp)
    G2F = Fp2
    if twist_degree == 6:
      non_residue_twist = curve_config[curve_name]['tower']['SNR_Fp2']
    else:
      raise NotImplementedError()
  elif G2_field == 'Fp':
    G2F = Fp
    if twist_degree == 6:
      non_residue_twist = curve_config[curve_name]['tower']['SNR_Fp']
    else:
      raise NotImplementedError()
  else:
    raise NotImplementedError()


  if twist == 'D_Twist':
    G2 = EllipticCurve(G2F, [0, b/G2F(non_residue_twist)])
  elif twist == 'M_Twist':
    G2 = EllipticCurve(G2F, [0, b*G2F(non_residue_twist)])
  else:
    raise ValueError('G2 must be a D_Twist or M_Twist but found ' + twist)


  cofactor = G2.order() // r


  out = {
    'curve': curve_name,
    'group': 'G2',
    'modulus': serialize_bigint(p),
    'order': serialize_bigint(r),
    'cofactor': serialize_bigint(cofactor),
    'form': form,
    'twist_degree': int(twist_degree),
    'twist': twist,
    'non_residue_fp': int(non_residue_fp),
    'G2_field': G2_field,
    'non_residue_twist': [int(coord) for coord in non_residue_twist] if isinstance(non_residue_twist, list) else int(non_residue_twist)
  }
  if form == 'short_weierstrass':
    out['a'] = serialize_bigint(a)
    out['b'] = serialize_bigint(b)


  vectors = []
  set_random_seed(seed)
  for i in progressbar(range(count)):
      v = {}
      P = G2.random_point()
      scalar = randrange(1 << scalarBits) if scalarBits else randrange(r)


      P *= cofactor # clear cofactor
      Q = scalar * P


      v['id'] = i
      if G2_field == 'Fp2':
        v['P'] = serialize_EC_Fp2(P)
        v['scalarBits'] = scalarBits if scalarBits else r.bit_length()
        v['scalar'] = serialize_bigint(scalar)
        v['Q'] = serialize_EC_Fp2(Q)
      elif G2_field == 'Fp':
        v['P'] = serialize_EC_Fp(P)
        v['scalarBits'] = scalarBits if scalarBits else r.bit_length()
        v['scalar'] = serialize_bigint(scalar)
        v['Q'] = serialize_EC_Fp(Q)
      vectors.append(v)


  out['vectors'] = vectors
  return out

In [3]:
def serialize_bigint(x):
  return '0x' + Integer(x).hex()

In [4]:
from sage.structure.proof.all import arithmetic

In [5]:
def progressbar(it, prefix="", size=60, file=sys.stdout):
  count = len(it)
  def show(j):
    x = int(size*j/count)
    file.write("%s[%s%s] %i/%i\r" % (prefix, "#"*x, "."*(size-x), j, count))
    file.flush()
  show(0)
  for i, item in enumerate(it):
    yield item
    show(i+1)
  file.write("\n")
  file.flush()

In [6]:
def serialize_EC_Fp2(P):
  (Px, Py, Pz) = P
  Px = vector(Px)
  Py = vector(Py)
  coords = {
    'x': {
      'c0': serialize_bigint(Px[0]),
      'c1': serialize_bigint(Px[1])
    },
    'y': {
      'c0': serialize_bigint(Py[0]),
      'c1': serialize_bigint(Py[1])
    }
  }
  return coords

In [17]:
def genAddG2(curve_name, curve_config, count, seed, scalarBits = None):
  p = curve_config[curve_name]['field']['modulus']
  r = curve_config[curve_name]['field']['order']
  form = curve_config[curve_name]['curve']['form']
  a = curve_config[curve_name]['curve']['a']
  b = curve_config[curve_name]['curve']['b']
  embedding_degree = curve_config[curve_name]['tower']['embedding_degree']
  twist_degree = curve_config[curve_name]['tower']['twist_degree']
  twist = curve_config[curve_name]['tower']['twist']


  G2_field_degree = embedding_degree // twist_degree
  G2_field = f'Fp{G2_field_degree}' if G2_field_degree > 1 else 'Fp'


  if G2_field_degree == 2:
    non_residue_fp = curve_config[curve_name]['tower']['QNR_Fp']
  elif G2_field_degree == 1:
    if twist_degree == 6:
      # Only for complete serialization
      non_residue_fp = curve_config[curve_name]['tower']['SNR_Fp']
    else:
      raise NotImplementedError()
  else:
    raise NotImplementedError()


  Fp = GF(p)
  K.<u> = PolynomialRing(Fp)


  if G2_field == 'Fp2':
    Fp2.<beta> = Fp.extension(u^2 - non_residue_fp)
    G2F = Fp2
    if twist_degree == 6:
      non_residue_twist = curve_config[curve_name]['tower']['SNR_Fp2']
    else:
      raise NotImplementedError()
  elif G2_field == 'Fp':
    G2F = Fp
    if twist_degree == 6:
      non_residue_twist = curve_config[curve_name]['tower']['SNR_Fp']
    else:
      raise NotImplementedError()
  else:
    raise NotImplementedError()


  if twist == 'D_Twist':
    G2 = EllipticCurve(G2F, [0, b/G2F(non_residue_twist)])
  elif twist == 'M_Twist':
    G2 = EllipticCurve(G2F, [0, b*G2F(non_residue_twist)])
  else:
    raise ValueError('G2 must be a D_Twist or M_Twist but found ' + twist)


  cofactor = G2.order() // r


  out = {
    'curve': curve_name,
    'group': 'G2',
    'modulus': serialize_bigint(p),
    'order': serialize_bigint(r),
    'cofactor': serialize_bigint(cofactor),
    'form': form,
    'twist_degree': int(twist_degree),
    'twist': twist,
    'non_residue_fp': int(non_residue_fp),
    'G2_field': G2_field,
    'non_residue_twist': [int(coord) for coord in non_residue_twist] if isinstance(non_residue_twist, list) else int(non_residue_twist)
  }
  if form == 'short_weierstrass':
    out['a'] = serialize_bigint(a)
    out['b'] = serialize_bigint(b)


  vectors = []
  set_random_seed(seed)
  for i in progressbar(range(count)):
      v = {}
      P1 = G2.random_point()
      P2 = -P1


      P1 *= cofactor # clear cofactor
      P2 *= cofactor

      Q = P1 + P1 + P1 


      v['id'] = i
      if G2_field == 'Fp2':
        v['P1'] = serialize_EC_Fp2(P1)
        v['P2'] = serialize_EC_Fp2(P2)
        v['Q'] = serialize_EC_Fp2(Q)
      elif G2_field == 'Fp':
        v['P1'] = serialize_EC_Fp2(P1)
        v['P2'] = serialize_EC_Fp2(P2)
        v['Q'] = serialize_EC_Fp(Q)
      vectors.append(v)


  out['vectors'] = vectors
  return out

In [18]:
genAddG2('BN254_Snarks',Curves, 1,3)

[############################################################] 1/1


{'curve': 'BN254_Snarks',
 'group': 'G2',
 'modulus': '0x30644e72e131a029b85045b68181585d97816a916871ca8d3c208c16d87cfd47',
 'order': '0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001',
 'cofactor': '0x30644e72e131a029b85045b68181585e06ceecda572a2489345f2299c0f9fa8d',
 'form': 'short_weierstrass',
 'twist_degree': 6,
 'twist': 'D_Twist',
 'non_residue_fp': -1,
 'G2_field': 'Fp2',
 'non_residue_twist': [9, 1],
 'a': '0x0',
 'b': '0x3',
 'vectors': [{'id': 0,
   'P1': {'x': {'c0': '0x1536b2e79d6f2116ce8f06e33cb7997612d2514df64a41e1e5fae866b1c0d779',
     'c1': '0x1f60381668f1c1d04cc11964dd3937c2ef0b11db4bbac18501c3ccd53ec87c6e'},
    'y': {'c0': '0x2dfe2cb093eafecb76994ca8cabb488d3171747c4e9fe6b3afc2123e31ac9c6f',
     'c1': '0x1bddcd8f468cc2657ef23699b3bc07a16c314cd67d46c8dbb9372985c2dcc8ec'}},
   'P2': {'x': {'c0': '0x1536b2e79d6f2116ce8f06e33cb7997612d2514df64a41e1e5fae866b1c0d779',
     'c1': '0x1f60381668f1c1d04cc11964dd3937c2ef0b11db4bbac18501c3ccd53ec87c6e'},
   

In [20]:
genScalarMulG2('BN254_Snarks', Curves, 2, 4)

[############################################################] 2/2


{'curve': 'BN254_Snarks',
 'group': 'G2',
 'modulus': '0x30644e72e131a029b85045b68181585d97816a916871ca8d3c208c16d87cfd47',
 'order': '0x30644e72e131a029b85045b68181585d2833e84879b9709143e1f593f0000001',
 'cofactor': '0x30644e72e131a029b85045b68181585e06ceecda572a2489345f2299c0f9fa8d',
 'form': 'short_weierstrass',
 'twist_degree': 6,
 'twist': 'D_Twist',
 'non_residue_fp': -1,
 'G2_field': 'Fp2',
 'non_residue_twist': [9, 1],
 'a': '0x0',
 'b': '0x3',
 'vectors': [{'id': 0,
   'P': {'x': {'c0': '0x2d8a913eeee7c28aa12c81a2dbf4e8a753b7745c4910254a0404f09c4f36d867',
     'c1': '0x686e06cae2b68521cfe51921e30ef9291eeee283f1b3b503a1c8c8f70b86017'},
    'y': {'c0': '0x1b1210574a3c68090fbaa2c595adcf2d5b0dadbaba73796d4f56f0c5aba15bfa',
     'c1': '0x179448931f57e3bff2dbbc2f394afa1ba523ec54ca8aabd344095d98ed99ce90'}},
   'scalarBits': 254,
   'scalar': '0x2f3ee6ad9ed78b75bf31c54d279ec79494a54793bacbffc439f186ece6f08d3f',
   'Q': {'x': {'c0': '0x24215fc5a8eb0a718c7d9bfe8cd257c661ad0951abf5c9905f